In [1]:
from db_utils import *
% matplotlib inline

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 24 days


In [2]:
# Get Pre 2015-03 data from pentaho
query = """
SELECT
sum(pageviews) as n,
timestamp, 
country_iso, 
project, 
access_method
FROM staging.pentahoviews05
WHERE is_spider = 0
AND is_automata = 0
AND project RLIKE 'wikipedia'
group by timestamp, country_name, project, access_method
"""

d =  query_analytics_store(query, {})

In [5]:
d.head()

,access_method,country_iso,country_name,is_automata,is_spider,is_zero,pageviews,project,referer,timestamp
0,b'desktop',b'CN',b'China',0,1,0,10000.0,b'pt.wiktionary',b'None',b'2013-04-01'
1,b'desktop',b'US',b'United States',0,0,0,26802000.0,b'en.wikipedia',b'Internal',b'2013-04-01'
2,b'desktop',b'US',b'United States',0,0,0,16386000.0,b'en.wikipedia',b'None',b'2013-04-01'
3,b'desktop',b'GR',b'Greece',0,0,0,475000.0,b'en.wikipedia',b'Internal',b'2013-04-01'
4,b'mobile web',b'JP',b'Japan',0,0,0,4065000.0,b'ja.wikipedia',b'None',b'2013-04-01'


In [36]:
str_dimensions = ['project', 'access_method', 'country_iso', 'timestamp']
d[str_dimensions] = d[str_dimensions].applymap(lambda x: x.decode("utf-8"))

In [37]:
# sampled data overlaps with hive data for 1 month (April 2015). We delete the month sampled data
d = d[d['timestamp'] != '2015-04-01']

In [38]:
# Get post 2015-03 data from hive

query = """
SET mapred.job.queue.name=priority;
SELECT
sum(view_count) as n,
CONCAT(year, '-', month, '-01') as timestamp, 
country_code as country_iso, 
project, 
access_method
FROM wmf.projectview_hourly
WHERE agent_type = 'user'
AND project RLIKE 'wikipedia'
AND YEAR = 2015
group by year, month, country_code, project, access_method;
"""

dn =  query_hive_ssh(query, 'forecasting_refresh')

In [39]:
df = pd.concat([d, dn], axis=0)

In [40]:
df.index = df['timestamp']

In [41]:
# Add checkpoint in case data gets lost
# Last Checkpoint: Sept 8
df.to_csv('/Users/ellerywulczyn/wmf/pageview_forecasting/checkpoint.csv')
#df = pd.read_csv('/Users/ellerywulczyn/wmf/pageview_forecasting/checkpoint.csv')

In [42]:
group_dimensions = ['project', 'access_method', 'country_iso',]
groups = df.groupby(group_dimensions)

In [43]:
cube = {}
for group in groups:
    colname = '/'.join(group[0])
    dg = group[1]
    dg.index = dg['timestamp']
    data = pd.Series(dg['n'])
    if data.sum() > 1000000:
        cube[colname] = data

In [51]:
len(cube)

2947

In [52]:
df_cube = pd.DataFrame(cube)

In [53]:
df_cube.tail()

,ab.wikipedia/desktop/US,ace.wikipedia/desktop/DE,ace.wikipedia/desktop/US,af.wikipedia/desktop/CN,af.wikipedia/desktop/DE,af.wikipedia/desktop/FR,af.wikipedia/desktop/NL,af.wikipedia/desktop/US,af.wikipedia/desktop/Unknown,af.wikipedia/desktop/ZA,...,zh.wikipedia/mobile web/NZ,zh.wikipedia/mobile web/PH,zh.wikipedia/mobile web/RU,zh.wikipedia/mobile web/SE,zh.wikipedia/mobile web/SG,zh.wikipedia/mobile web/TH,zh.wikipedia/mobile web/TW,zh.wikipedia/mobile web/US,zh.wikipedia/mobile web/Unknown,zh.wikipedia/mobile web/VN
2015-5-01,43223,115412,86982,23824,147660,75855,28311,905726,NaN,639185,...,247468,61713,123301,79066,1680911,213355,50131187,11381063,NaN,187900
2015-6-01,31862,101400,62851,15260,128717,46763,28707,647306,NaN,231775,...,241866,62424,108630,68455,1649751,205275,48374828,9233813,NaN,169015
2015-7-01,16575,25791,42379,21373,83345,183840,16432,423695,NaN,285268,...,253019,71775,79768,77384,1750086,274022,50026437,7687829,NaN,200521
2015-8-01,23292,21199,46863,11946,107698,101138,39297,478241,NaN,768345,...,277718,76324,115577,81268,1837474,305238,54495466,8661747,NaN,197986
2015-9-01,11772,9601,24844,37239,41219,23524,11453,321281,NaN,348148,...,135512,33343,45945,41326,845696,117668,22278646,3406463,NaN,87123


In [54]:
df_cube['YearMonth'] = pd.to_datetime(df_cube.index)

In [55]:
df_codes = pd.read_csv('./app/data/country_codes.csv')[['ISO 3166-1 2 Letter Code', 'Common Name' ]]
codes_dict = dict(tuple(x) for x in df_codes.values)
codes_dict['Unknown'] = 'Unknown'

df_valid_cols = [c for c in df_cube.columns if len(c.split('/')) == 3 and c.split('/')[2] in codes_dict]
df_valid_cols.append('YearMonth')
df_cube = df_cube[df_valid_cols]

def replace_ISO_with_country(c,codes_dict):
    if c == 'YearMonth':
        return c
    t = c.split('/')
    t[2] = codes_dict[t[2]]
    return '/'.join(t)

df_cube.columns = [replace_ISO_with_country(c,codes_dict) for c in df_cube.columns]

In [57]:
df_cube.to_csv('/Users/ellerywulczyn/wmf/pageview_forecasting/app/data/cube.csv', index = False)

In [56]:
df_cube.shape

(30, 2867)